# API 🪢

En este jupyter notebook se encuentra todo el proceso de exportación de datos y su posterior limpieza utilizando una API. La API escogida ha sido World Data Bank.

*Condideraciones*

De nuevo, sólo se tendrán en cuenta los 11 países seleccionados para el estudio: España, Francia, Italia, Grecia, Turquia, USA, China, India, Mexico, Nueva Zelanda, Reino Unido.

## Índice 📎

1. Importación de librerías y funciones
2. Extracción de datos
3. Limpieza de datos
4. Exportación del dataset final

## 1. Importación de librerías y funciones 📚

In [1]:
import pandas as pd
import os
import requests
import json
import datetime

Para extraer y solicitar la información de las diversas bases de datos de World Data Bank nos importaremos la siguiente interfaz la cual contiene todos los metadatos completos y algunas funciones convenientes para la búsqueda y recuperación de información: 

In [2]:
import wbdata

## 2. Extracción de datos  🎣

Para utilizar la API de World Bank primero es necesario localizar el identificador de la base de datos que quieres utilizar para ello:

In [3]:
databases = requests.get("http://api.worldbank.org/v2/sources?per_page=100&format=json")
databases = databases.json()
#databases

for x in databases[1]:
    print(x["id"],x["name"])

1 Doing Business
2 World Development Indicators
3 Worldwide Governance Indicators
5 Subnational Malnutrition Database
6 International Debt Statistics
11 Africa Development Indicators
12 Education Statistics
13 Enterprise Surveys
14 Gender Statistics
15 Global Economic Monitor
16 Health Nutrition and Population Statistics
18 IDA Results Measurement System
19 Millennium Development Goals
20 Quarterly Public Sector Debt
22 Quarterly External Debt Statistics SDDS
23 Quarterly External Debt Statistics GDDS
24 Poverty and Equity
25 Jobs
27 Global Economic Prospects
28 Global Financial Inclusion
29 The Atlas of Social Protection: Indicators of Resilience and Equity
30 Exporter Dynamics Database – Indicators at Country-Year Level
31 Country Policy and Institutional Assessment
32 Global Financial Development
33 G20 Financial Inclusion Indicators
34 Global Partnership for Education
35 Sustainable Energy for All
36 Statistical Capacity Indicators
37 LAC Equity Lab
38 Subnational Poverty
39 Health

En nuestro caso para hallar los datos referentes a la esperanza de vida utilizaremos la base de datos "World Development Indicators" cuyo número ID es el 2. Lo siguiente será identificar el ID del indicador que contiene la información en materia a la esperanza de vida, para ello creamos un diccionario y buscamos el indicador.

In [4]:
ID = "2"
indicadores = requests.get(f"http://api.worldbank.org/v2/indicator?format=json&source={ID}&per_page=1500")
indicadores = indicadores.json()
wd_indicadores = dict()

#Diccionario de indicadores

for x in indicadores[1]:
    key = x["id"]
    value = x["name"]
    wd_indicadores[key] = value
#print (wd_indicators)

El ID del indicador es ➡️ SP.DYN.LE00.IN (esperanza de vida al nacer), SP.DYN.LE00.FE.IN (esperanza de vida al nacer mujeres) y SP.DYN.LE00.MA.IN (esperanza de vida al nacer varones)

Lo siguiente será determinar la lista de países (en nuestro caso la tomaremos entera y luego procederemos a limpiarla) y el periodo de tiempo (se escogeran los datos referidos al año 2019, ya que son los más actuales).

In [5]:
pais = requests.get("http://api.worldbank.org/v2/country?format=json&per_page=250")
pais = pais.json()
#pais
pais_id = []
pais_nombre = []

for x in pais[1]:
    p_id = (x["id"])
    p_n = (x["id"], x["name"])
    pais_id.append(p_id)
    pais_nombre.append(p_n)
    
#print(pais_id)
#print(pais_nombre)

Estos serán los datos y parámetros que indicaremos para crear nuestro DataFrame:

In [6]:
indic = {"SP.DYN.LE00.IN":wd_indicadores.get("SP.DYN.LE00.IN"),"SP.DYN.LE00.FE.IN":wd_indicadores.get("SP.DYN.LE00.FE.IN"),"SP.DYN.LE00.MA.IN":wd_indicadores.get("SP.DYN.LE00.MA.IN")}
lista_paises = pais_id
periodo = datetime.datetime(2019, 1, 1), datetime.datetime(2019, 1, 1)
ID = "2"

Por último obtenemos los datos de la API y construimos el DataFrame.

*Se empleará la interfaz wbdata importada al principio para ello*

In [7]:
#get_dataframe(indicators, country, data_date, freq, source, convert_date, keep_levels, cache)
data = wbdata.get_dataframe(indicators = indic,
                            source = ID,
                            country = lista_paises, 
                            data_date = periodo)

In [8]:
data.head()

,"Life expectancy at birth, total (years)","Life expectancy at birth, female (years)","Life expectancy at birth, male (years)"
country,,,
Aruba,76.293,78.641,73.768
Afghanistan,64.833,66.388,63.382
Angola,61.147,64.039,58.416
Albania,78.573,80.201,77.002
Andorra,NaN,NaN,NaN


In [9]:
#Establecemos un índice y dejamos country como columna
data = data.reset_index()

In [10]:
data.head()

,country,"Life expectancy at birth, total (years)","Life expectancy at birth, female (years)","Life expectancy at birth, male (years)"
0,Aruba,76.293,78.641,73.768
1,Afghanistan,64.833,66.388,63.382
2,Angola,61.147,64.039,58.416
3,Albania,78.573,80.201,77.002
4,Andorra,NaN,NaN,NaN


## 3. Limpieza de datos 🧹

In [11]:
data

,country,"Life expectancy at birth, total (years)","Life expectancy at birth, female (years)","Life expectancy at birth, male (years)"
0,Aruba,76.293000,78.641000,73.768000
1,Afghanistan,64.833000,66.388000,63.382000
2,Angola,61.147000,64.039000,58.416000
3,Albania,78.573000,80.201000,77.002000
4,Andorra,NaN,NaN,NaN
...,...,...,...,...
215,Sub-Saharan Africa,61.627443,63.410146,59.859330
216,Small states,69.770842,71.623533,68.097811
217,Sao Tome and Principe,70.385000,72.814000,67.980000
218,Suriname,71.682000,75.063000,68.468000


Renombramos las columnas para facilitar su legibilidad

In [12]:
columnas = ["country","life_expectancy","life_expectancy_f","life_expectancy_m"]

In [13]:
data.columns = columnas
data

,country,life_expectancy,life_expectancy_f,life_expectancy_m
0,Aruba,76.293000,78.641000,73.768000
1,Afghanistan,64.833000,66.388000,63.382000
2,Angola,61.147000,64.039000,58.416000
3,Albania,78.573000,80.201000,77.002000
4,Andorra,NaN,NaN,NaN
...,...,...,...,...
215,Sub-Saharan Africa,61.627443,63.410146,59.859330
216,Small states,69.770842,71.623533,68.097811
217,Sao Tome and Principe,70.385000,72.814000,67.980000
218,Suriname,71.682000,75.063000,68.468000


Procedemos a quedarnos con los 11 países seleccionados: España, Francia, Italia, Grecia, Turquía, Estados Unidos, China, India, México, Nueva Zelanda y Reino Unido. 

In [14]:
España = data[data["country"]=="Spain"]
Francia = data[data["country"]=="France"]
Italia = data[data["country"]=="Italy"]
Grecia = data[data["country"]=="Greece"]
Turquia = data[data["country"]=="Turkey"]
USA = data[data["country"]=="United States"]
China = data[data["country"]=="China"]
India = data[data["country"]=="India"]
Mexico = data[data["country"]=="Mexico"]
Nueva_Zelanda = data[data["country"]=="New Zealand"]
Reino_Unido = data[data["country"]=="United Kingdom"]

paises = [España, Francia, Italia, Grecia, Turquia, USA, China, India, Mexico, Nueva_Zelanda, Reino_Unido]

In [15]:
data = pd.concat(paises)#Filtramos los países seleccionados
data

,country,life_expectancy,life_expectancy_f,life_expectancy_m
68,Spain,83.485366,86.200,80.900
75,France,82.578049,85.600,79.700
114,Italy,83.197561,85.400,81.100
87,Greece,81.939024,84.500,79.500
38,China,76.912000,79.244,74.761
107,India,69.656000,70.950,68.464
152,Mexico,75.054000,77.879,72.203
178,New Zealand,81.707317,83.500,80.000
79,United Kingdom,81.204878,83.100,79.400


In [16]:
data2 = data.reset_index(drop=True)#Reestructuramos el índice
data2

,country,life_expectancy,life_expectancy_f,life_expectancy_m
0,Spain,83.485366,86.200,80.900
1,France,82.578049,85.600,79.700
2,Italy,83.197561,85.400,81.100
3,Greece,81.939024,84.500,79.500
4,China,76.912000,79.244,74.761
5,India,69.656000,70.950,68.464
6,Mexico,75.054000,77.879,72.203
7,New Zealand,81.707317,83.500,80.000
8,United Kingdom,81.204878,83.100,79.400


## Exportación del dataset final 🚀

In [17]:
vida_data = data2
vida_data.to_csv("./vida_data.csv") #Exportación del DataFrame